<a href="https://colab.research.google.com/github/ShounakDas101/AIML_Hari/blob/main/MNIST_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
# clearing cuda cache memory
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [3]:
from google.colab import drive
drive.mount("/content/drive")
# dataset directory
# this directory contains all the datasets related for ML4SCI tests.
os.listdir("/content/drive/MyDrive/MyAIML/input")

Mounted at /content/drive


['fashion-mnist_test.csv',
 'fashion-mnist_train.csv.zip',
 'fashion-mnist_test.csv.zip',
 'fashion-mnist_train.csv']

In [4]:
# Read the CSV file into a Pandas DataFrame
train = pd.read_csv("/content/drive/MyDrive/MyAIML/input/fashion-mnist_train.csv", dtype = np.float32)
test = pd.read_csv("/content/drive/MyDrive/MyAIML/input/fashion-mnist_test.csv", dtype = np.float32)

In [5]:
#set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
y_train = train['label'] # extracts the column labeled "label" from the train DataFrame.
# drops the "label" column from the train DataFrame along the horizontal axis (axis=1).
# /255 is to mormalize data
X_train = train.drop(labels = ['label'], axis = 1)/255
X_test = test.drop(labels = ['label'], axis = 1)/255


features_train, features_test, targets_train, targets_test = train_test_split(X_train,
                                                                             y_train,
                                                                             test_size = 0.2,
                                                                             random_state = 42)


# put the data to gpu(the device we set)
featuresTrain = torch.from_numpy(features_train.values).to(device)
targetsTrain = torch.from_numpy(targets_train.values).type(torch.LongTensor).to(device) # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.from_numpy(features_test.values).to(device)
targetsTest = torch.from_numpy(targets_test.values).type(torch.LongTensor).to(device) # data type is long


X_Test = torch.from_numpy(X_test.values)

batch_size = 100
n_iters = 14000
num_epochs = n_iters / (len(features_train) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)



train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

# data loader: set shuffle = true
train_loader = DataLoader(train, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = True)


class ANNModel(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ANNModel, self).__init__()

        # Linear function 1: 784 --> 150
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # Non-linearity 1
        self.relu1 = nn.ReLU()

        # Linear function 2: 150 --> 150
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 2
        self.tanh2 = nn.Tanh()

        # Linear function 3: 150 --> 150
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 3
        self.elu3 = nn.ELU()

        # Linear function 4 (readout): 150 --> 10
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)

        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.tanh2(out)

        # Linear function 2
        out = self.fc3(out)
        # Non-linearity 2
        out = self.elu3(out)

        # Linear function 4 (readout)
        out = self.fc4(out)
        return out

input_dim = 28*28
hidden_dim = 28*28*2 + 1
output_dim = 10

# put the model to gpu
model = ANNModel(input_dim, hidden_dim, output_dim).to(device)

error = nn.CrossEntropyLoss()

learning_rate = 0.02
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)




# ANN model training
count = 0
loss_list = []
iteration_list = []
accuracy_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        #Variable --> cuda
        train = Variable(images.view(-1, 28*28)).cuda()
        labels = Variable(labels).cuda()

        optimizer.zero_grad()

        outputs = model(train)

        loss = error(outputs, labels)

        loss.backward()

        optimizer.step()

        count += 1

        if count % 50 == 0:

            correct = 0
            total = 0

            for images, labels in test_loader:

                test = Variable(images.view(-1, 28*28))

                outputs = model(test)

                predicted = torch.max(outputs.data, 1)[1]

                total += len(labels)

                correct += (predicted == labels).sum()

            accuracy = 100 * correct / float(total)

            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        if count % 500 == 0:
            # Print Loss
            print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data, accuracy))




output = model(test)

predicted = torch.max(outputs.data, 1)[1]

predicted_numpy = predicted.data.cpu().numpy()

output = pd.Series(predicted_numpy, name="Label")

submission = pd.concat([pd.Series(range(1,10001), name = "ImageId"), output], axis = 1)

29
Iteration: 500  Loss: 0.6163017749786377  Accuracy: 74.14166259765625 %
Iteration: 1000  Loss: 0.6156318783760071  Accuracy: 79.81666564941406 %
Iteration: 1500  Loss: 0.5288286805152893  Accuracy: 82.03333282470703 %
Iteration: 2000  Loss: 0.42523711919784546  Accuracy: 83.51666259765625 %
Iteration: 2500  Loss: 0.5225428342819214  Accuracy: 83.80833435058594 %
Iteration: 3000  Loss: 0.49648529291152954  Accuracy: 83.79166412353516 %
Iteration: 3500  Loss: 0.20887860655784607  Accuracy: 84.83333587646484 %
Iteration: 4000  Loss: 0.4253695607185364  Accuracy: 82.38333129882812 %
Iteration: 4500  Loss: 0.3630848228931427  Accuracy: 85.24166870117188 %
Iteration: 5000  Loss: 0.4081645607948303  Accuracy: 84.93333435058594 %
Iteration: 5500  Loss: 0.3039710521697998  Accuracy: 86.36666870117188 %
Iteration: 6000  Loss: 0.33326759934425354  Accuracy: 86.59166717529297 %
Iteration: 6500  Loss: 0.3308435380458832  Accuracy: 86.67499542236328 %
Iteration: 7000  Loss: 0.4387902319431305  Ac